<a href="https://colab.research.google.com/github/DataSavvyYT/experiments/blob/main/1_llm_finetune/1_finetuned_gemma_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os

# Update 'python3.x' to match the current Colab version (usually python3.11 or python3.12 in late 2025)
env_path = "/content/drive/MyDrive/unsloth_env/lib/python3.12/site-packages"

if env_path not in sys.path:
    sys.path.append(env_path)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
# 1. Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b", # You can swap this for 'unsloth/mistral-7b'
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# 2. Add LoRA adapters (The "Sticky Notes")
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
DATASET_PATH = "json"
# ----------------------------
# Load dataset
# Expect JSONL with fields: instruction, input, output
# ----------------------------
dataset = load_dataset(
    DATASET_PATH,
    data_files={
        "train": "/content/drive/MyDrive/data/promotion/train.jsonl",
        "validation": "/content/drive/MyDrive/data/promotion/validation.jsonl"
    }
)

In [ ]:
dataset['train'].to_pandas().head()

In [ ]:
# Formatting function to create supervised prompts
SYSTEM_PREFIX = "You are an analyst that predicts promotion effectiveness based on campaign details."
INSTR_TEMPLATE = """<system>
{system}
</system>
<instruction>
{instruction}
</instruction>
<input>
{inp}
</input>
<output>
{out}
</output>"""

In [ ]:
def format_example(example):
    instruction = example.get("instruction", "Predict promotion effectiveness.")
    inp = example.get("input", "")
    out = example.get("output", "")
    # SFTTrainer learns to map input -> output; include output as labels portion
    return INSTR_TEMPLATE.format(system=SYSTEM_PREFIX, instruction=instruction, inp=inp, out=out)

def map_fn(batch):
    texts = [format_example(ex) for ex in batch]
    return {"text": texts}

train_ds = dataset["train"].map(lambda ex: {"text": format_example(ex)})
eval_ds = dataset["validation"].map(lambda ex: {"text": format_example(ex)})

In [ ]:
train_ds.to_pandas().head()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set this to 60 for a quick demo, or 1000 for real training
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()


In [ ]:
def predict_effectiveness(description: str) -> str:
    prompt = f"""<system>
{SYSTEM_PREFIX}
</system>
<instruction>
Predict promotion effectiveness as one of: "effective", "not effective", or a probability between 0 and 1.
</instruction>
<input>
{description}
</input>
<output>
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # naive parse: take last line after <output>
    return text.split("<output>")[-1].strip()



In [ ]:
print(predict_effectiveness("Campaign: Diwali Sale; Channel: Email; Budget: 5 Lakh INR; Audience: Returning; Discount: 10%; Duration: 5 days; Past CTR: 2.8%"))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

true_labels = []
predictions = []

print("Starting evaluation on validation dataset...")
for i, example in enumerate(eval_ds):
    if i % 10 == 0:
        print(f"Processing example {i}/{len(eval_ds)}")

    campaign_description = example["input"]
    true_label = example["output"].strip().lower()

    # Get prediction from the model
    predicted_raw = predict_effectiveness(campaign_description)
    predicted_text = predicted_raw.strip().lower()

    predicted_label = ""
    try:
        # Check if the output is a probability
        predicted_prob = float(predicted_text)
        # Assuming a threshold of 0.5 for classification if a probability is returned
        predicted_label = "effective" if predicted_prob > 0.5 else "not effective"
    except ValueError:
        # If not a probability, assume it's directly the label string
        predicted_label = predicted_text

    true_labels.append(true_label)
    predictions.append(predicted_label)

print("\nEvaluation complete.")

# Ensure we have labels for classification_report to avoid errors
unique_labels = sorted(list(set(true_labels + predictions)))

if not true_labels:
    print("No true labels found in the validation dataset. Cannot compute metrics.")
elif not unique_labels:
    print("No labels (true or predicted) found. Cannot compute metrics.")
elif len(unique_labels) == 1:
    # If only one class is present in true or predicted labels, accuracy_score works,
    # but classification_report might struggle. Handle gracefully.
    print(f"Only one label ('{unique_labels[0]}') present in true or predicted labels.")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")
    # No classification report if only one label is present
else:
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")

    print("\nClassification Report:")
    # Use zero_division=0 to prevent warnings/errors if a class has no samples/predictions
    print(classification_report(true_labels, predictions, labels=unique_labels, zero_division=0))